In [ ]:
import numpy as np
import nengo
from nengo_gui.ipython import IPythonViz

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize=(10, 6))

## Trajectory detection

Basic idea:

- We know the trajectory through time,
  and use this to make continuous predictions of the trajectory.
- When the actual trajectory matches our predictions, we advance time.
- We can interrogate how far along the trajectory we are.

In [ ]:
import nengo.utils.numpy as npext

def predict(x):
    # Simple trajectory that just goes through 4 discrete states
    if x <= 0.25:
        return np.array([0, 0, 1, 1]).astype(float)
    elif x <= 0.5:
        return np.array([0, 1, 0, 1]).astype(float)
    elif x <= 0.75:
        return np.array([1, 0, 1, 0]).astype(float)
    else:
        return np.array([1, 1, 0, 0]).astype(float)


def observe(t):
    # Same trajectory, but at various points in time
    if t <= 0.5:
        return np.array([0, 0, 1, 1]).astype(float)
    elif t <= 0.6:
        return np.array([0, 1, 0, 1]).astype(float)
    elif t <= 0.7:
        return np.array([1, 0, 1, 0]).astype(float)
    else:
        return np.array([1, 1, 0, 0]).astype(float)


def dist(v1, v2):
    v1 = np.atleast_2d(v1)
    v2 = np.atleast_2d(v2)
    dots = np.dot(v1, v2.T)

    # Zero-norm vectors should return zero, so avoid divide-by-zero error
    eps = np.nextafter(0, 1)  # smallest float above zero
    v1norm = np.maximum(npext.norm(v1, axis=1, keepdims=True), eps)
    v2norm = np.maximum(npext.norm(v2, axis=1, keepdims=True), eps)
    dots /= v1norm
    dots /= v2norm.T
    return dots


# Go from time 0 to 2
t = np.linspace(0, 1, num=200)
# State
x = 0.
dt = 0.02
x_hist = []
for tt in t:
    # Make a prediction, get an observation
    pred = predict(x)
    obs = observe(tt)

    # Find similarity with cosine distance
    d = dist(pred, obs)[0, 0] > 0.9
    # Increment x
    x += d * dt
    x = np.clip(x, 0, 1)
    x_hist.append(x)

plt.plot(t, x_hist)

## Trajectory detection in Nengo

We do basically the inverse of DMPs;
we're trying to recover the ramp function
given the trajectory that is decoded from the ramp.
If we do this for all DMPs representing syllables,
then the syllable we just heard is
the one with the maximum ramp function.

In [ ]:
def forced_func(x):
    # x ramps up from 0 to 1
    if x < 0.4:
        return [0, 0, 1]
    elif x < 0.8:
        return [0, 1, 0]
    else:
        return [1, 0, 0]

def non_forced_func(x):
    if x < 0.4:
        return [1, 0, 0]
    elif x < 0.8:
        return [0, 1, 0]
    else:
        return [0, 0, 1]

def ff_inv(func, scale=0.15):
    def forced_func_inv(x):
        actual_x = np.array([x[0]] + func(x[0]))
        dot = np.dot(x, actual_x) ** 2
        return x[0] + dot*scale
    return forced_func_inv


with nengo.Network() as net:
    kick = nengo.Node(lambda t: 1 if t < 0.2 else 0)

    # linearly increasing system with an oscillatory biased input
    oscillator = nengo.Ensemble(500, dimensions=2, radius=0.01)
    # recurrent connection
    nengo.Connection(oscillator, oscillator,
                     transform=np.eye(2) + np.array([[1, -1], [1, 1]]))

    # input pulse to start off the first integrator
    nengo.Connection(kick, oscillator, transform=np.ones((2, 1)))

    # make a slow ramp up
    forcing_func = nengo.Ensemble(1000, dimensions=1)

    # make first dimension of forcing function ensemble an integrator
    nengo.Connection(forcing_func[0], forcing_func[0], synapse=0.1)

    # set up the input to the integrating first dimensions 
    nengo.Connection(oscillator, forcing_func[0], 
                     transform=.2, 
                     function=lambda x: x[0]+.5)

    # connect up an inhibitory signal to prevent the forcing function
    # from interfering with the DMP state getting to it's starting point
    nengo.Connection(kick, forcing_func.neurons, 
                     transform=-np.ones((forcing_func.n_neurons, 1)), synapse=.01)

    # the decoded function
    forced = nengo.Ensemble(90, dimensions=3)
    nengo.Connection(forcing_func, forced, function=forced_func)

    # recover `x` ramp from forced function
    forced_inv = nengo.Ensemble(500, dimensions=4, n_eval_points=5000)
    # first dimension is a recurrent connection,
    # advancing x or not depending on the input observation
    nengo.Connection(forced_inv, forced_inv[0],
                     function=ff_inv(forced_func), synapse=0.05)
    # last three dimensions are getting input observations
    nengo.Connection(forced, forced_inv[1:])

    # Reset with the kick
    nengo.Connection(kick, forced_inv.neurons, synapse=.01,
                     transform=-np.ones((forced_inv.n_neurons, 1)))

    # try a different function to make sure that function won't work
    non_forced_inv = nengo.Ensemble(500, dimensions=4, n_eval_points=10000)
    nengo.Connection(non_forced_inv, non_forced_inv[0],
                     function=ff_inv(non_forced_func), synapse=0.05)
    nengo.Connection(forced, non_forced_inv[1:])
    nengo.Connection(kick, non_forced_inv.neurons, synapse=.01,
                     transform=-np.ones((non_forced_inv.n_neurons, 1)))

    p_ff = nengo.Probe(forcing_func, synapse=0.01)
    p_forced = nengo.Probe(forced, synapse=0.01)
    p_finv = nengo.Probe(forced_inv, synapse=0.01)
    p_nfinv = nengo.Probe(non_forced_inv, synapse=0.01)

In [ ]:
# IPythonViz(net, cfg='ff.cfg')
sim = nengo.Simulator(net)
sim.run(1.)

t = sim.trange()
plt.figure()
plt.plot(t, sim.data[p_ff])
plt.figure()
plt.plot(t, sim.data[p_forced])
plt.figure()
plt.plot(t, sim.data[p_finv])
plt.figure()
plt.plot(t, sim.data[p_nfinv])